In [58]:
import pandas as pd
import numpy as np
import os
import torch
import torch.nn.functional as F 
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

train = pd.read_excel("C:/Users/amend/Desktop/hand_Data_test_BIG.xlsx")
train.replace(r'^\s*$', np.nan, regex=True, inplace=True)
train.dropna(inplace=True)


test = pd.read_excel("C:/Users/amend/Desktop/hand_Data_train_f.xlsx")
test.replace(r'^\s*$', np.nan, regex=True, inplace=True)
test.dropna(inplace=True)
print(len(test.columns))




65


In [60]:
device = (
    "cpu"
)
print (device)
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.layers=nn.Sequential(
            nn.Linear(len(train.columns)-2,512),
            nn.ReLU(),
            nn.Linear(512,364),
            nn.ReLU(),
            nn.Linear(364,128),
            nn.ReLU(),
            nn.Linear(128,64),
            nn.Dropout(0.2),
            nn.ReLU(),
            nn.Linear(64,32),
            nn.ReLU(),
            nn.Linear(32,6)
        )
    def forward(self,x):
        logits=self.layers(x)

        return logits


cpu


In [61]:
model=NeuralNetwork()

In [62]:
import torch.optim as optim
valid_loader= torch.utils.data.DataLoader(train, batch_size=4, shuffle=True)
classes=("call","stop","left","right","dislike","like")

In [63]:
loss_fn=torch.nn.CrossEntropyLoss()
opt=optim.Adam(model.parameters(), lr=1e-3)

In [64]:
from torch.utils.data import Dataset

class GestureDataset(Dataset):
    def __init__(self, dataframe):
        self.data = dataframe
        # Map 'gesture' and 'Hand_Id' to numerical values
        self.label_mapping = {gesture: idx for idx, gesture in enumerate(self.data['gesture'].unique())}
        self.hand_mapping = {'Left': 0, 'Right': 1}  # Map Hand_Id to numerical values
        self.data['gesture'] = self.data['gesture'].map(self.label_mapping)
        self.data['Hand_Id'] = self.data['Hand_Id'].map(self.hand_mapping)
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        # Include 'Hand_Id' as part of the input features
        hand_id = self.data.iloc[idx, 1]  # Get the 'Hand_Id' value
        features = self.data.iloc[idx, 2:].values.astype(float)  # All columns except 'gesture'
        features = torch.tensor(features, dtype=torch.float32)
        #features = torch.cat((torch.tensor([hand_id], dtype=torch.float32), features))  # Include 'Hand_Id'

        # Labels (gesture)
        label = torch.tensor(self.data.iloc[idx, 0], dtype=torch.long)  # 'gesture' column
                
        return features, label
tr=GestureDataset(train)
train_loder= torch.utils.data.DataLoader(tr, batch_size=40, shuffle=True)
test=GestureDataset(test)
test_loader=torch.utils.data.DataLoader(test)


In [65]:
def acc(y_pred,y):
    _,pred=torch.max(y_pred,1)
    yes=(pred==y).sum().item()
    return yes/y.size(0)

In [66]:
from sklearn.preprocessing import StandardScaler
for epoch in range (70):
    ls=0
    model.train()
    for batch,(X,y) in enumerate(train_loder):
        X,y=X.to(device),y.to(device)

        mean=torch.nanmean(X)   
        X=torch.nan_to_num(X,nan=mean)
        #print(X)
        # print(f"Max value in X: {X.max()}")
        # print(f"Min value in X: {X.min()}")
        # print(f"Max value in y: {y.max()}")
        # print(f"Min value in y: {y.min()}")
       # X = (X - X.mean()) / X.std()

        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        log=model(X)
        print("NaNs in model output:", torch.isnan(log).sum())
        #print(y)
        loss=loss_fn(log,y)
        print("NaNs in loss:", torch.isnan(loss).sum())
        opt.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        print("batch number",batch)
        print(acc(log,y))
        print("acc",loss.item())
        opt.step()
        ls+=loss.item()


NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 0
0.175
acc 1.7928367853164673
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 1
0.2
acc 1.7827532291412354
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 2
0.275
acc 1.7685420513153076
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 3
0.15
acc 1.7966750860214233
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 4
0.2
acc 1.7848379611968994
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 5
0.1
acc 1.7910531759262085
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 6
0.075
acc 1.7983394861221313
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 7
0.25
acc 1.7730834484100342
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 8
0.25
acc 1.7906081676483154
NaNs in model output: tensor(0)
NaNs in loss: tensor(0)
batch number 9
0.175
acc 1.7858664989471436
NaNs in m

In [67]:
ls
print(ls/50)

0.1118966530286707


In [69]:
model.eval()

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=63, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=364, bias=True)
    (3): ReLU()
    (4): Linear(in_features=364, out_features=128, bias=True)
    (5): ReLU()
    (6): Linear(in_features=128, out_features=64, bias=True)
    (7): Dropout(p=0.2, inplace=False)
    (8): ReLU()
    (9): Linear(in_features=64, out_features=32, bias=True)
    (10): ReLU()
    (11): Linear(in_features=32, out_features=6, bias=True)
  )
)

In [71]:
sum=0
i=0
with torch.no_grad():
    for _,(x,y) in enumerate(test_loader):
        x,y=x.to(device),y.to(device)
        y_pred=model(x)
        accuracy=acc(y_pred,y)
        print("loss",loss_fn(y_pred,y))
        sum+=accuracy
        i+=1
        print(accuracy)
print("tot",(sum/i))

loss tensor(1.1921e-07)
1.0
loss tensor(0.0002)
1.0
loss tensor(2.2196)
0.0
loss tensor(0.0023)
1.0
loss tensor(0.0123)
1.0
loss tensor(4.4822e-05)
1.0
loss tensor(0.0037)
1.0
loss tensor(0.0017)
1.0
loss tensor(6.1398)
0.0
loss tensor(0.0003)
1.0
loss tensor(0.)
1.0
loss tensor(0.0011)
1.0
loss tensor(7.5457e-05)
1.0
loss tensor(1.0610e-05)
1.0
loss tensor(0.0038)
1.0
loss tensor(0.)
1.0
loss tensor(7.9509e-05)
1.0
loss tensor(7.3907e-05)
1.0
loss tensor(0.)
1.0
loss tensor(0.0001)
1.0
loss tensor(0.0006)
1.0
loss tensor(9.8943e-06)
1.0
loss tensor(7.1526e-07)
1.0
loss tensor(0.0002)
1.0
loss tensor(4.7206e-05)
1.0
loss tensor(3.2987)
0.0
loss tensor(0.0005)
1.0
loss tensor(0.0057)
1.0
loss tensor(0.0001)
1.0
loss tensor(1.5497e-06)
1.0
loss tensor(3.0994e-06)
1.0
loss tensor(4.3391e-05)
1.0
loss tensor(1.0490e-05)
1.0
loss tensor(3.5763e-06)
1.0
loss tensor(0.0001)
1.0
loss tensor(5.9605e-07)
1.0
loss tensor(0.0015)
1.0
loss tensor(3.0351)
0.0
loss tensor(0.0004)
1.0
loss tensor(0.00

In [73]:
#saving model
torch.save(model,'model_f.pt')
